# IMDB Point Prediction with Random Forests

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split

sns.set(color_codes=True)
sns.set(style="ticks")

data_frame_original = pd.read_csv("movie_metadata.csv")
#data_frame.loc[data_frame['imdb_score'] >= 7.5, 'imdb_score'].size / data_frame['imdb_score'].count()


In [ ]:
data_frame = data_frame_original.copy()
data_frame = data_frame.drop('genres',axis=1);
data_frame = data_frame.drop('plot_keywords',axis=1);
data_frame = data_frame.drop('movie_imdb_link', axis=1)
data_frame = data_frame.drop('movie_title',axis=1)
data_frame.loc[data_frame['imdb_score'] < 7.5, 'imdb_score'] = 0
data_frame.loc[data_frame['imdb_score'] >= 7.5, 'imdb_score'] = 1
data_frame


In [ ]:

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import LabelEncoder

for column_name in data_frame.columns:
    le = LabelEncoder()
    column = data_frame[column_name]
    try:
        if column_name == 'plot_keywords' or column_name == 'genres':
            mlb = MultiLabelBinarizer()
            keywords = list()
            column.fillna('UNKNOWN')
            column.apply(lambda x: keywords.append(x.split('|')) if x is not np.nan else keywords.append(['UNKNOWN']))
            binarized = mlb.fit_transform(keywords)
            plt.imshow(binarized)
            class_values = pd.Series(list(binarized))
            data_frame[column_name].update(class_values)

        else:
            normalized_column = column.fillna(0) if column.dtype.kind in 'biufc' else column.fillna('UNKNOWN')
            class_values = normalized_column if column.dtype.kind in 'biufc' else pd.Series(list(le.fit_transform(normalized_column))) 
            data_frame[column_name].update(class_values)
    except:
        pass

data_frame

In [ ]:
# Figures for representing some features
sns.set(font_scale = 2)
g = sns.jointplot('gross', 'imdb_score', size=12, data=data_frame, kind="kde", color="#10275F")
plt.subplots_adjust(top=0.55)
g.fig.suptitle('KDE of Imdb_Score and Gross', size=20, weight='bold')
sns.set(font_scale = 1)
# This is different from the kaggle kernel result(as in shape)

In [ ]:
# Showing the pearson correlation of features
with sns.plotting_context(font_scale=1.25):
    f, ax = plt.subplots(figsize=(20, 20))
    plt.title('Pearson Correlation of Movie Features', {'weight': 'bold', 'size': 20})
    # plot_keywords features are encoded as array which requires more care to plot in this way
    
    sns.heatmap(data_frame.astype(float).corr(), linewidths=0.25, vmax=1.0, square=True, annot=True)

Create training and test datasets with .75 probab.

In [ ]:

# Load scikit's random forest classifier library
from sklearn.ensemble import RandomForestClassifier

data_frame['is_train'] = np.random.uniform(0,1,len(data_frame)) <=0.75

train,test = data_frame[data_frame['is_train'] == True],data_frame[data_frame['is_train'] == False]
#print(len(train))
#print(len(test))





Select features to use in the training, and the target class to predict

In [ ]:
## select the features
features = data_frame.columns
train_feat = features[features != 'imdb_score']
train_results = train.imdb_score

Fit the model to RandomForestClassifier with the training data

Get the results of the model for the test class and give the precision

In [ ]:
print(len(set(data_frame.actor_1_name)))
clf = RandomForestClassifier(n_jobs=-1,bootstrap=True, n_estimators=10)

clf.fit(train[train_feat], train_results);


prediction = clf.predict(test[train_feat])

print(prediction)
result =  test.imdb_score - prediction
print(len(result[result !=0]) / len(result))
pd.crosstab(test['imdb_score'], prediction, rownames=['Actual Imbd'], colnames=['Predicted Imdb'])
list(zip(train[features], clf.feature_importances_))